In [1]:
from app.utilities import *
import ibis
from ibis import _

con = ibis.duckdb.connect("duck.db", extensions=['httpfs', 'spatial', 'h3'])
set_secrets(con) # s3 credentials
set_aws_secrets(con)

#con.raw_sql("SET memory_limit = '20GB';")
#con.raw_sql("set threads=40;")

gbif = con.read_parquet("s3://cboettig/gbif/2024-10-01/**")

# can/should we add explicit spatial index to gbif first?  

2024-12-01 20:00:44.842 No runtime found, using MemoryCacheStorageManager


In [2]:
(con
 .read_geo("/vsicurl/https://data.source.coop/cboettig/us-boundaries/mappinginequality.json")
 .to_parquet("s3://cboettig/gbif/mappinginequality.parquet")
)

# can/should we add explicit spatial index to mappinginequality polygons first? 
# would local duckdb table version be even better/faster? 


In [4]:
## select cities from the list we haven't yet written (allows resume).
import minio
import streamlit as st
from pathlib import Path



minio_key = st.secrets["MINIO_KEY"]
minio_secret = st.secrets["MINIO_SECRET"]
mc = minio.Minio("minio.carlboettiger.info", minio_key, minio_secret)
obj = mc.list_objects("cboettig", "gbif/redlined/", recursive=True)

finished_cities = [str(Path(i.object_name).stem) for i in obj if not i.is_dir]
mappinginequality = con.read_parquet("s3://cboettig/gbif/mappinginequality.parquet")

all_cities = mappinginequality.select(_.city).distinct().order_by(_.city).execute()["city"]
remaining_cities = set(all_cities) - set(finished_cities)


In [ ]:
%%time 

## And here we go, long-running loop over each city
for city in remaining_cities:
    print(city)
    gdf = (mappinginequality
           .filter(_.city == city)
           .mutate(area = _.geom.area())
#           .agg(geom = _.geom.unary_union())
    )
    
    bounds =  gdf.execute().total_bounds
    points = (gbif
           .filter(_.decimallongitude >= bounds[0], 
                   _.decimallongitude < bounds[2], 
                   _.decimallatitude >= bounds[1], 
                   _.decimallatitude < bounds[3])
             )
                  
    (gdf
     .join(points, gdf.geom.intersects(points.geom))
     .to_parquet(f"s3://cboettig/gbif/redlined/{city}.parquet")
    )


New Orleans


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Norfolk


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Schenectady


...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

York


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Dedham


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
%%time

## Testing

(con
  .read_parquet("s3://cboettig/gbif/redlined/*")
  .filter(_.city == "Los Angeles")
  .group_by(_.grade)
  .agg(n = _.count(),
       area = _.area.sum())
  .mutate(density = _.n /_.area)
  .order_by(_.density.desc())
  .execute()
)


In [ ]:
%%time 

## mean & sd within grade

(con
 .read_parquet("s3://cboettig/gbif/redlined/*")
 .filter(_.city == "Los Angeles")
 .group_by(_.area_id, _.grade)
 .agg(n = _.count(),
      area = _.area.sum())
 .mutate(density = _.n /_.area)
 .group_by(_.grade)
 .agg(mean = _.density.mean(),
        sd = _.density.std())
 .order_by(_.mean.desc())
 .execute()
)

In [ ]:
## Overture
overture = con.read_parquet('s3://overturemaps-us-west-2/release/2024-11-13.0/theme=divisions/type=division_area/*', filename=True, hive_partitioning=1)
usa = overture.filter(_.subtype=="country").filter(_.country == "US").select(_.geometry).execute()
ca = (overture
       .filter(_.country == "US", _.subtype == "region")
       .select('region', 'geometry')
       .filter(_.region == "US-CA")
       .execute()
      )

In [ ]:

## export in gdal formats?  not working?
## FIXME test duckdb gdal writes?
bucket = "cboettig/gbif"
dest2 = "cache/tmp.geojson"
query = ibis.to_sql(sel)
#con.raw_sql(f"COPY ({query}) TO 's3://{bucket}/{dest2}'  WITH (FORMAT GDAL, DRIVER 'FlatGeoBuf');")
#con.raw_sql(f"COPY ({query}) TO 's3://{bucket}/{dest2}'  WITH  (FORMAT GDAL, DRIVER 'GeoJSON', LAYER_CREATION_OPTIONS 'WRITE_BBOX=YES');")
